In [1]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis\\Research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis'

In [16]:
dir_path = "Dataset/Model_selection"

In [5]:
from ml_Project.utils.common import create_directories

In [17]:
def create_dir(dir_path):
    create_directories([dir_path])

In [21]:
create_dir(dir_path)

[2024-08-15 13:58:14,422: INFO: common: created directory at: Dataset/Model_selection]


In [12]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

In [13]:
train_data_path = "Dataset/Data_transformation/train.csv"
test_data_path = "Dataset/Data_transformation/test.csv"
target_col = "Diabetes_012"
dir_path = "Dataset/Model_selection"

In [36]:
import os
import json

class ModelTrainer:
    def __init__(self, train_data_path, test_data_path, target_col, dir_path):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.target_col = target_col
        self.dir_path = dir_path

    def create_dir(self):
        create_directories([self.dir_path])

    def load_data(self):
        train_data = pd.read_csv(self.train_data_path)
        test_data = pd.read_csv(self.test_data_path)
        return train_data, test_data

    def preprocess_data(self, train_data, test_data):
        X_train = train_data.drop([self.target_col], axis=1)
        X_test = test_data.drop([self.target_col], axis=1)
        y_train = train_data[[self.target_col]]
        y_test = test_data[[self.target_col]]

        # One-hot encode the target variable
        y_train = to_categorical(y_train, num_classes=3)
        y_test = to_categorical(y_test, num_classes=3)

        return X_train, X_test, y_train, y_test

    def build_custom_model(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape)

        # Branch 1
        x1 = Dense(128, activation="relu")(inputs)
        x1 = Dense(128, activation="relu")(x1)
        x1 = Dense(64, activation="relu")(x1)

        # Branch 2
        x2 = Dense(128, activation="relu")(inputs)
        x2 = Dense(128, activation="relu")(x2)
        x2 = Dense(64, activation="relu")(x2)

        # Branch 3
        x3 = Dense(128, activation="relu")(inputs)
        x3 = Dense(128, activation="relu")(x3)
        x3 = Dense(64, activation="relu")(x3)

        # Branch 4
        x4 = Dense(128, activation="relu")(inputs)
        x4 = Dense(128, activation="relu")(x4)
        x4 = Dense(64, activation="relu")(x4)

        # Combine branches
        combined = Concatenate()([x1, x2, x3, x4])

        # Fully connected layers
        x = Dense(256, activation="relu")(combined)
        x = Dense(256, activation="relu")(x)
        outputs = Dense(3, activation="softmax")(x)

        # Build model
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

        return model

    def train(self):

        self.create_dir()

        # Load data
        train_data, test_data = self.load_data()

        # Preprocess data
        X_train, X_test, y_train, y_test = self.preprocess_data(train_data, test_data)

        # Build and train model
        model = self.build_custom_model((X_train.shape[1], ))

        # Train the model and save history
        history = model.fit(X_train, y_train, epochs=30, batch_size=128, validation_data=(X_test, y_test))

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test)
        print(f"Model Accuracy: {accuracy:.4f}")

        # Save model weights
        model.save_weights(os.path.join(self.dir_path, "model_mlp.h5"))

        # Save history
        history_dict = history.history
        history_path = os.path.join(self.dir_path, "training_history.json")
        with open(history_path, 'w') as f:
            json.dump(history_dict, f)

        print(f"Training history saved to {history_path}")

In [37]:
trainer = ModelTrainer(train_data_path, test_data_path, target_col)
trainer.train()

Epoch 1/30
2849/2849 [==============================] - 11s 3ms/step - loss: 0.9560 - accuracy: 0.5161 - val_loss: 0.9066 - val_accuracy: 0.5560
Epoch 2/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.8561 - accuracy: 0.5868 - val_loss: 0.8349 - val_accuracy: 0.6032
Epoch 3/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.7591 - accuracy: 0.6411 - val_loss: 0.7253 - val_accuracy: 0.6635
Epoch 4/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.6867 - accuracy: 0.6806 - val_loss: 0.6440 - val_accuracy: 0.7030
Epoch 5/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.6240 - accuracy: 0.7168 - val_loss: 0.5924 - val_accuracy: 0.7336
Epoch 6/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.5665 - accuracy: 0.7455 - val_loss: 0.5560 - val_accuracy: 0.7540
Epoch 7/30
2849/2849 [==============================] - 8s 3ms/step - loss: 0.5190 - accuracy: 0.7700 - val_loss: 0.5114 - val_accuracy